In [1]:
print("hello")

hello


In [16]:
import pandas as pd
import re

In [17]:
data = {
    'studentID' : [101,101,101,101,101,
                   102,102,102,102,
                   103,103,103,103,103,
                   104,104,104,104,10
                   ],
    'attendance_date': [
        '2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05',
        '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05',
        '2024-03-05', '2024-03-06', '2024-03-07', '2024-03-08', '2024-03-09',
         '2024-03-01', '2024-03-02', '2024-03-03', '2024-03-04', '2024-03-05'
    ],
    'status': ['Absent', 'Absent', 'Absent' , 'Absent', 'Present',
               'Absent', 'Absent', 'Absent', 'Absent',
               'Absent', 'Absent', 'Absent', 'Absent', 'Absent',
               'Present', 'Present', 'Absent', 'Present', 'Present'
               ]
}

In [18]:
df = pd.DataFrame(data)
df['attendance_date'] = pd.to_datetime(df['attendance_date'])
df

,studentID,attendance_date,status
0,101,2024-03-01,Absent
1,101,2024-03-02,Absent
2,101,2024-03-03,Absent
3,101,2024-03-04,Absent
4,101,2024-03-05,Present
5,102,2024-03-02,Absent
6,102,2024-03-03,Absent
7,102,2024-03-04,Absent
8,102,2024-03-05,Absent
9,103,2024-03-05,Absent


In [19]:
df['attendance_date'] = pd.to_datetime(df['attendance_date'])
df = df.sort_values(by=['studentID', 'attendance_date'])

absent_streaks = []
for student_id, group in df.groupby('studentID'):
    group = group.reset_index(drop=True)
    streak_start = None
    streak_count = 0

    for i in range(len(group)):
        if group['status'][i] == 'Absent':
            if streak_start is None:
                streak_start = group['attendance_date'][i]
            streak_count += 1
        else:
            if streak_count > 3:
                absent_streaks.append((student_id, streak_start, group['attendance_date'][i - 1], streak_count))
            streak_start = None
            streak_count = 0

    if streak_count > 3:
        absent_streaks.append((student_id, streak_start, group['attendance_date'].iloc[-1], streak_count))

result_df = pd.DataFrame(absent_streaks, columns=['student_id', 'absence_start_date', 'absence_end_date', 'total_absent_days'])
latest_streaks = result_df.loc[result_df.groupby('student_id')['absence_start_date'].idxmax()]
print(latest_streaks)

   student_id absence_start_date absence_end_date  total_absent_days
0         101         2024-03-01       2024-03-04                  4
1         102         2024-03-02       2024-03-05                  4
2         103         2024-03-05       2024-03-09                  5


In [25]:
students_data = {
    'student_id': [101, 102, 103, 104],
    'student_name': ['Alice', 'Bob', 'Charlie', 'David'],
    'parent_email': ['alice@gmail.com', 'bob@gmail', 'charlie@domain.com', 'david@domain.com']
}

students_df = pd.DataFrame(students_data)
merged_df = pd.merge(result_df, students_df, on='student_id')

def is_valid_email(email):
    if '@' not in email:
        return False
    local_part, domain = email.split('@', 1)
    if not re.match(r'^[a-zA-Z_][a-zA-Z0-9_]*$', local_part):
        return False
    if domain != 'domain.com' and not domain.endswith('.com'):
        return False
    return True

merged_df['email'] = merged_df['parent_email'].apply(lambda x: x if is_valid_email(x) else None)
merged_df['msg'] = merged_df.apply(
    lambda row: f"Dear Parent, your child {row['student_name']} was absent from {row['absence_start_date'].date()} to {row['absence_end_date'].date()} for {row['total_absent_days']} days. Please ensure their attendance improves."
    if row['email'] is not None else None,
    axis=1
)
final_df = merged_df[['student_id', 'student_name', 'absence_start_date','absence_end_date','total_absent_days','email','msg']]
final_df

,student_id,student_name,absence_start_date,absence_end_date,total_absent_days,email,msg
0,101,Alice,2024-03-01,2024-03-04,4,alice@gmail.com,"Dear Parent, your child Alice was absent from ..."
1,102,Bob,2024-03-02,2024-03-05,4,None,None
2,103,Charlie,2024-03-05,2024-03-09,5,charlie@domain.com,"Dear Parent, your child Charlie was absent fro..."
